In [1]:
""" 
1. Использовать dataset MovieLens
2. Построить рекомендации ( предсказываем оценку)
на фичах:
■ TF IDF на тегах и жанрах
■ Средние оценки median, variance, etc.) пользователя и фильма
3. Оценить RMSE на тестовой выборке
"""
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm  #  интеллектуальный индикатор выполнения - 
                                # просто оберните любую итерацию с помощью tqdm (iterable)

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

%matplotlib inline




In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# задача - мы должны понять, какую оценку пользователь поставит фильму (целевая переменная - rating)
# 1. Для этого мы изучим: 
#     1.1. фильмам каких жанров человек ставит оценки
#           (отфильтровать по пользователю, выделить фильмы, которым он поставил оценки, узнать жанры этих фильмов)
#     1.2. какие оценки ставит пользователь данным жанрам
#     1.3. как он размечает фильмы тегами 
#     1.4. расставим веса жанров и тегов методом tfidf 
# 2. соединяем таблицы фильмов и рейтингов, чтобы понимать, какому фильм какая оценка поставлена (для тестовой выборки)
# 3. сформируем линейную регрессию и сделаем прогноз
# 4. найдем ошибку RMSE

In [6]:
# movies_tags = movies.join(tags.set_index('movieId'), on='movieId')
movies_tags = movies.merge(tags, on='movieId')
movies_tags

,movieId,title,genres,userId,tag,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,474,pixar,1137206825
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,567,fun,1525286013
3,2,Jumanji (1995),Adventure|Children|Fantasy,62,fantasy,1528843929
4,2,Jumanji (1995),Adventure|Children|Fantasy,62,magic board game,1528843932
...,...,...,...,...,...,...
3678,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,62,star wars,1528934552
3679,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,anime,1537098582
3680,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,comedy,1537098587
3681,193565,Gintama: The Movie (2010),Action|Animation|Comedy|Sci-Fi,184,gintama,1537098603


In [7]:
movies_tags_ratings = movies_tags.merge(ratings.set_index('movieId'), on='userId')

movies_tags_ratings.head()

,movieId,title,genres,userId,tag,timestamp_x,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.0,1122227329
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.0,1122227549
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.5,1122227343
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,5.0,1120568496
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,1139045764,4.0,1120568169


In [8]:
new_df = movies_tags_ratings.pop('timestamp_x')


In [9]:
movies_tags_ratings

,movieId,title,genres,userId,tag,rating,timestamp_y
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,1122227329
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,1122227549
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.5,1122227343
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,5.0,1120568496
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0,1120568169
...,...,...,...,...,...,...,...
4626861,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5,1515194979
4626862,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.0,1533519112
4626863,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,5.0,1529413865
4626864,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5,1533519140


In [10]:
movies_tags_ratings_ = movies_tags_ratings.pop('timestamp_y')

In [11]:
# Либо удалить столбцы можно было через команду del или так:
# movies_tags_ratings.drop(['timestamp_x', 'timestamp_y'], axis=1).head()

In [12]:
movies_tags_ratings

,movieId,title,genres,userId,tag,rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.5
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,5.0
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,336,pixar,4.0
...,...,...,...,...,...,...
4626861,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5
4626862,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.0
4626863,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,5.0
4626864,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5


In [13]:
user_462 = movies_tags_ratings[movies_tags_ratings.userId==462]
user_462.sample(10)

,movieId,title,genres,userId,tag,rating
4626155,152711,Who Killed Chea Vichea? (2010),Documentary,462,murder,4.0
4626297,152711,Who Killed Chea Vichea? (2010),Documentary,462,murder,4.0
4626116,152711,Who Killed Chea Vichea? (2010),Documentary,462,murder,3.0
4625445,152711,Who Killed Chea Vichea? (2010),Documentary,462,crime,2.5
4626329,152711,Who Killed Chea Vichea? (2010),Documentary,462,murder,4.0
4624803,152711,Who Killed Chea Vichea? (2010),Documentary,462,Cambodia,4.0
4624857,152711,Who Killed Chea Vichea? (2010),Documentary,462,Cambodia,1.5
4625501,152711,Who Killed Chea Vichea? (2010),Documentary,462,human rights,1.5
4626151,152711,Who Killed Chea Vichea? (2010),Documentary,462,murder,3.5
4626494,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,3.5


In [14]:
def change_string(s):
    """
    Функция принимает строку и возвращает список слов, разденных в столбце символом '|', 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [15]:
# прогоним через функцию строки столбца жанров 
movie_genres = [change_string(g) for g in user_462.genres.values]

In [16]:
# обучим модель для понимания, каким жанром принадлежит каждый их фильмов 
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)
X_train_counts.toarray()

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int64)

In [17]:
# обучим модель, чтобы придать жанрам различные веса 
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [18]:
# реализовав метод, видим, что вес фильмов изменился в зависимости от его жанров (чем реже жанр, тем выше вес)
X_train_tfidf.toarray()

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]])

In [19]:
# уникальных значений в столбце тегов не так много (в сравнении со всем фреймом)
user_462.tag.nunique()

5

In [20]:
# некоторе теги встречаются заметно чаще остальных: это значит, что метод tfidf способен разнести веса от высоких к низким
user_462['tag'].value_counts() 

crime           455
murder          455
procedural      455
Cambodia        455
human rights    455
Name: tag, dtype: int64

In [21]:
def change_string_tag(s):
    """
    Функция принимает строку и возвращает список слов, разденных в столбце пробелом, 
    которые склеивает (.join()) через пробел
    """
    return ' '.join(s.replace(' ', '').replace('-', '').split(' '))

In [22]:
# прогоним через функцию строки столбца жанров 
movie_tags = [change_string_tag(g) for g in user_462.tag.values]

In [23]:
# обучим модель для понимания, какими тегами описывается каждый их фильмов 
# для экономии места не будем трансформировать разреженную матрицу в обычный массив
X_train_tags = count_vect.fit_transform(movie_tags)

In [24]:
# Другие возможности для экономии пространства:
# 1) заменить все редкие теги одним значением (например, other):

# movies_tags_ratings.loc[movies_tags_ratings['tag'].value_counts()<100, 'tag'] = other
# или так:
# DataFrame['column_name'] = numpy.where(condition, new_value, DataFrame.column_name)
# еще можно так:
# movies_tags_ratings['tag'].replace(movies_tags_ratings['tag'].value_counts()<100, 'other')

# 2) параметрами ограничить число фич, создаваемых нашими трансформерами
# 3) мы оставили матрицу разреженной. В sklearn есть модели, которые умеют с ними работать, например, Naive Bayes
# 4) предварительно можно понизить размерность через TruncatedSVD, PCA

# from sklearn.decomposition import PCA
# from sklearn.decomposition import TruncatedSVD
# pca = TruncatedSVD(n_components = 1)
# XPCAreduced = pca.fit_transform(transpose(movies_tags_ratings))

In [25]:
# посмотрим, есть ли пустые значения столбца тегов: нет 
movies_tags_ratings['tag'].isnull().any()

False

In [26]:
# обучим модель, чтобы придать тегам различные веса 
tfidf_transformer = TfidfTransformer()
X_train_tfidf_tags = tfidf_transformer.fit_transform(X_train_tags)

In [34]:
print(X_train_tfidf.shape)
print(X_train_tfidf_tags.shape)
user_462['X_tfidf_genres'] = X_train_tfidf
user_462['X_tfidf_tags'] = X_train_tfidf_tags


(2275, 1)
(2275, 5)


C:\Users\zlatt\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\zlatt\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [32]:
user_462

,movieId,title,genres,userId,tag,rating,X_tfidf_genres,X_tfidf_tags
4624591,152711,Who Killed Chea Vichea? (2010),Documentary,462,Cambodia,1.5,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
4624592,152711,Who Killed Chea Vichea? (2010),Documentary,462,Cambodia,3.0,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
4624593,152711,Who Killed Chea Vichea? (2010),Documentary,462,Cambodia,3.5,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
4624594,152711,Who Killed Chea Vichea? (2010),Documentary,462,Cambodia,4.0,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
4624595,152711,Who Killed Chea Vichea? (2010),Documentary,462,Cambodia,3.0,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
...,...,...,...,...,...,...,...,...
4626861,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
4626862,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.0,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
4626863,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,5.0,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."
4626864,152711,Who Killed Chea Vichea? (2010),Documentary,462,procedural,4.5,"(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ...","(0, 0)\t1.0\n (1, 0)\t1.0\n (2, 0)\t1.0\n ..."


In [29]:
# Удалим столбцы жанров и тегов с тем, чтобы следом добавить признаки, созданные трансформерами 
# user_462.drop(['genres', 'tag'], axis=1).head()